# CRISPRi Workflow - Work in Progress
##### Goals of this package: improve the validation process for gold standards, define a unified pipeline for increased reproducibility, provide an accessible format for analysis.

Ideas for future:
- compare the outcomes of using DESeq2 vs. the simple normalization and t-test methods employed below, using CRISPhieRmix or other packages for False Discovery Rate calculation, and trying different types of raw count normalization (two I have already defined in this package - possibly the most relevant methods)
- adapt/include functions for other types of library-based functional screens (TnSeq,etc)

<div class="alert alert-block alert-warning">
Immediate needs:

- This package is a work in progress and there are several issues that need handling. Please find 'TODO's' throughout this tutorial workflow to see what is missing or needs improvement. I will also create Issues in GitHub that others can contribute to.
- Most important: while I have successfully tested these functions on sample data, there is no testing folder! I recommend pytest for a more thorough and robust confirmation that things are working correctly. Developers should set up a folder wherein every .py file starts with test_ and every defined function in those files starts with test_ and use the assert command in those functions to assure that calculations happen as expected. To run the tests, just type 'pytest testing_folder' on terminal.
</div>

# Installation and Set-up

##### To install for development and use, clone the GitHub repository to your local computer. Go to terminal and enter:

```bash
git clone https://github.com/evfaust/gene_feature_exploration.git
```

##### To develop the package, you might need to build it. In terminal in a working directory that is the repository you just cloned, enter the following. You may also be able to download the .tar.gz from releases on GitHub once released.

```bash
python -m build
pip install dist/crispri_data_analysis-0.0.1.tar.gz
```

It may also be helpful to work in a conda environment that has all the dependencies needed for this package. Download the .yml environment file from GitHub and enter in terminal:

```bash
conda env create -f environment.yml -n CRISPRi
```

To activate the conda environment and begin using these dependencies, enter the following in terminal. Any packages you install in an activated conda environment will be added to that environment.

```bash
conda activate CRISPRi
```

In [6]:
"""Import necessary modules"""
from CRISPRi_data_analysis.counter import *
from CRISPRi_data_analysis.normalize import *
from CRISPRi_data_analysis.gene_fitness import *
import subprocess
import glob

# Processing Data

##### Here we will run a bash script to unzip, merge, convert to fasta, and align your fastq input files to the reference gRNA library. Expect alignment to take some time. It is also possible to run this script directly from the terminal window.

**note, you will need to input the correct file paths

In [23]:
"""Run the next cell to see help documentation for what information is required."""

help_out = subprocess.run(["CRISPRi_data_analysis/process_data.sh",
                         "-h"])
print(help_out)

Process raw paired-end sequence files (ending with _R1.fastq.gz or _R2.fastq.gz) from a CRISPRi screen
(unzip, merge, convert to fasta, align, count reads, and combine samples into single dataframe all_counts.tsv).

Example with minimum inputs: ./process_data.sh -r /path/to/raw/seqs -d /path/to/library_database -a /path/for/out_alignments -c /path/for/out_counts -w wash_control_name
Syntax: scriptTemplate [-h|r|o|d|i|m|t]
options:
h     Print this help.
r     Enter the folder path storing raw paired-end .fastq.gz (zipped) sequence files. Ensure paired files have the same prefix and end with '_R1.fastq.gz' or _R2.fastq.gz'. 
Merged .fasta sequence files will be stored here.
d     Enter the database .fasta file as alignment reference. Must contain the reverse complements of guides.
o     Enter the desired output folder for storing alignment .tsv files. Script will create folder if it does not exist.
i     Enter the percent identity for alignment as a fraction. (Default is 0.9.)
m     Ent

In [7]:
""" Supply the necessary inputs """

# Required folder and library file paths:
raw_seq_folder = 'test_raw_seq' # must contain all fastq.gz files to be analyzed. merged fastq files will go here
database_library = 'test_raw_seq/database_22B1.fasta' # reference library file in fasta format 
out_folder = 'OUT_ALIGN_COUNTS' # all alignment and count outputs will go here

# Optional parameters with defaults:
percent_id_align = 0.9
min_seq_length = 1
target_cov_align = 1

In [34]:
""" Run this cell to process the data """

processing_result = subprocess.run(["CRISPRi_data_analysis/process_data.sh",
                                    "-r", raw_seq_folder,
                                    "-d", database_library,
                                    "-o", out_folder,
                                    "-i", str(percent_id_align),
                                    "-m", str(min_seq_length),
                                    "-t", str(target_cov_align)])
print(processing_result)

vsearch v2.28.1_macos_x86_64, 16.0GB RAM, 12 cores
https://github.com/torognes/vsearch

Merging reads 100%
   1000000  Pairs
    936493  Merged (93.6%)
     63507  Not merged (6.4%)

Pairs that failed merging due to various reasons:
      1121  too few kmers found on same diagonal
        22  multiple potential alignments
      8224  too many differences
     50049  alignment score too low, or score drop too high
         1  overlap too short
      4090  staggered read pairs

Statistics of all reads:
    150.00  Mean read length

Statistics of merged reads:
    204.68  Mean fragment length
      5.72  Standard deviation of fragment length
      0.15  Mean expected error in forward sequences
      0.24  Mean expected error in reverse sequences
      0.20  Mean expected error in merged sequences
      0.44  Mean observed errors in merged region of forward sequences
      0.58  Mean observed errors in merged region of reverse sequences
      1.02  Mean observed errors in merged region
vse

hi


 100%
Counting k-mers 100%
Creating k-mer index 100%
Searching 100%
Matching unique query sequences: 804532 of 936493 (85.91%)
vsearch v2.28.1_macos_x86_64, 16.0GB RAM, 12 cores
https://github.com/torognes/vsearch

Merging reads

hi2


 100%
   1000000  Pairs
    936493  Merged (93.6%)
     63507  Not merged (6.4%)

Pairs that failed merging due to various reasons:
      1121  too few kmers found on same diagonal
        22  multiple potential alignments
      8224  too many differences
     50049  alignment score too low, or score drop too high
         1  overlap too short
      4090  staggered read pairs

Statistics of all reads:
    150.00  Mean read length

Statistics of merged reads:
    204.68  Mean fragment length
      5.72  Standard deviation of fragment length
      0.15  Mean expected error in forward sequences
      0.24  Mean expected error in reverse sequences
      0.20  Mean expected error in merged sequences
      0.44  Mean observed errors in merged region of forward sequences
      0.58  Mean observed errors in merged region of reverse sequences
      1.02  Mean observed errors in merged region
vsearch v2.28.1_macos_x86_64, 16.0GB RAM, 12 cores
https://github.com/torognes/vsearch

Reading file /Us

hi


 100%
Counting k-mers 100%
Creating k-mer index 100%
Searching

hi2
CompletedProcess(args=['/Users/evelynfaust/Desktop/ORNL/CRISPRi_data_analysis/CRISPRi_data_analysis/process_data.sh', '-r', '/Users/evelynfaust/Desktop/ORNL/CRISPRi_data_analysis/test_raw_seq', '-d', '/Users/evelynfaust/Desktop/ORNL/CRISPRi_data_analysis/test_raw_seq/database_22B1.fasta', '-o', '/Users/evelynfaust/Desktop/ORNL/CRISPRi_data_analysis/NEW_TEST_ALIGN_OUT', '-c', '/Users/evelynfaust/Desktop/ORNL/CRISPRi_data_analysis/NEW_TEST_COUNT_OUT', '-i', '0.9', '-m', '1', '-t', '1'], returncode=0)


 100%
Matching unique query sequences: 804532 of 936493 (85.91%)


<div class="alert alert-block alert-warning">
<b>TODO:</b>

- store statistics/diagnostics of merging and aligning somewhere

- options for strandedness, top_hits_only, etc?

- options for filtering by average Q value or trimming non-overlapping sections

- user fields options? also put in help for output

- output counts into the same .tsv file column at a time
</div>

# Count Abundance and Combine Data

##### Here we will count raw abundance from alignment .tsv files

In [8]:
""" Supply the necessary inputs """

# Required 
out_counts_filename = 'all_counts.tsv' # output file name of master .tsv file that will include all sample counts (will go in previously defined out_folder)
wash_control = 'pWGA128_escapeControl' # for diagnostic

# Optional parameters with defaults:
id_filter = 100.0

In [4]:
""" Run this cell to count the abundance of each guide and run wash control diagnostic """

# Get a list of all files in the out_folder that end with _aligned.tsv
files = glob.glob(out_folder + '/*_aligned.tsv')

# Iterate abundance counting function all alignment files you have
for file in files:
    print("Processing file:", file)
    count_abundance(file = file, # .tsv alignment file from vsearch
                    design = database_library, # same database library reference from alignment
                    out_file = file.replace('_aligned.tsv', '_counts.tsv'),
                    wash_control = wash_control,
                    id_filter = id_filter)


Processing file: /Users/evelynfaust/Desktop/ORNL/CRISPRi_data_analysis/NEW_TEST_ALIGN_OUT/22B2_aligned.tsv
Processing file: /Users/evelynfaust/Desktop/ORNL/CRISPRi_data_analysis/NEW_TEST_ALIGN_OUT/22B1_aligned.tsv


In [7]:
""" Run this cell to consolidate all the count files into one master .tsv file"""

combine_count_files(out_folder, out_counts_filename)

<div class="alert alert-block alert-warning">
<b>TODO:</b> create script to combine forward and reverse strand counts for the same guide...come up with standard naming structure for the 'design' column that everyone can follow
</div>

# Normalize Counts

##### Here we will normalize the count data. Note: this normalization step can be skipped if using DESEq2 or PyDESeq2, where normalization is automatic. Be aware that DESeq2 automatically filters out genes with 0-counts, extremely high counts, and low mean normalized counts.

>    Median Ratios Method ('median_ratios'): This method accounts for the factors of sequencing depth and RNA composition, and is the method incorporated into DESeq2. Good for gene count comparisons between samples and for DE analysis; NOT for within sample comparisons. Source: https://github.com/hbctraining/DGE_workshop/blob/master/lessons/02_DGE_count_normalization.md

>    Total Reads Method ('total_reads'): Counts scaled by total number of reads. Accounts for seqencing depth. Not recommended for differential expression analysis.


In [9]:
""" Supply the necessary inputs """

# Required
all_counts_file = out_folder + '/' + out_counts_filename # this builds on the previous step's output file, with a guide design name column called 'design'
normalization_method = 'median_ratios' # options: 'median_ratios', 'total_reads', '' for no normalization

In [11]:
""" Run this cell to normalize counts """

# Set up the counts dataframe
all_counts = set_up_counts_df(all_counts_file)

# Normalize - these function add normalized count columns to the all_counts dataframe
if normalization_method == 'median_ratios':
    median_ratio_normalize(all_counts)
    print("counts normalized with median ratios")

elif normalization_method == 'total_reads':
    total_reads_normalize(all_counts)
    print("counts normalized with total reads")

else:
    print("counts not normalized")

counts normalized with median ratios


In [12]:
"""Save normalized counts as additional columns to the counts file"""

all_counts.to_csv(all_counts_file, sep = '\t') # note - this overwrites the old all_counts_file.tsv (keeping original and normalized counts)

# Compare Relative Gene Fitness

##### Here we calculate log2FC values between control and experimental conditions (combining replicates) and perform 2-tailed heteroscedastic t-tests (assumes variances between samples are unequal).

> Note: an alternative to this approach is to use DESeq2 or PyDESeq2 on raw counts. Be aware that DESeq2 automatically normalizes the counts and filters out genes with 0-counts, extremely high counts, and low mean normalized counts.

In [13]:
""" Provide Necessary Inputs """
# Required:
all_norm_counts = all_counts  # an indexed pandas dataframe with normalized counts for each guide (row) in each sample (column)
sample_prefixes = ['22B2'] # list of prefixes for sample column names in the counts df
control_prefix = '22B1' # prefix for control condition column names

> Note: The above inputs assume you want to compare guide abundance between each sample and the control condition. If there is more than 1 control condition or you want to compare between samples, change the inputs above to reflect that (sample vs control is irrelevant so long as you know what you are comparing)

In [14]:
""" Impute 0.5 for 0-counts to avoid log errors """

all_norm_counts.replace(0, 0.5, inplace=True)

In [15]:
""" Run this cell to generate log2FC and p-values and populate into columns """

compare_abundance(counts_df = all_norm_counts, sample_prefixes = sample_prefixes, control_prefix = control_prefix)

/Users/evelynfaust/anaconda3/envs/CRISPRi/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:573: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/evelynfaust/anaconda3/envs/CRISPRi/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:573: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/evelynfaust/anaconda3/envs/CRISPRi/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:573: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/evelynfaust/anaconda3/envs/

# Calculate False Discovery Rates

##### Now would be an appropriate time to calculate false discovery rates (more conservative p-values), owing to the fact that you are likely testing for the effect of knocking down thousands of genes (called multiple hypothesis testing). Furthermore, your library likely employed 10+ different guide RNAs to target individual genes (they likely have different knockdown effects), raising the number of hypotheses tested.

> With a simple p value cut-off at 0.05 indicating you accept a false positive rate of 5%, 5% of those many guides might be a substantially large number of false positives. 

> To do this, you might try using the package CRISPhieRmix

<div class="alert alert-block alert-warning">
<b>TODO:</b> evaluate CRISPhieRmix and write R script. Inputs are simple: 'normalized log fold changes of the individual gene targeting guides, along with the associated genes (in the same order as the log fold changes), with the option of including the log fold changes of the negative control guides'

https://genomebiology.biomedcentral.com/articles/10.1186/s13059-018-1538-6
</div>

# Calculate Alternative Fitness Scores

##### Now would be an appropriate time to calculate alternative relative gene fitness scores (or, continue the analysis with log2 fold change values). You may consider using absolute average, FIPER, and BAGEL methods.

<div class="alert alert-block alert-warning">
<b>TODO:</b> write .py file with functions that calculate a few different alternative fitness scores that people commonly use
</div>

# Perform Gene Set Enrichment Analysis

##### This depends on the level of annotation available for your organism. Using KEGG pathways (as opposed to Gene Ontology) will likely give the best chance of finding enriched sets of related genes. 

> I like using DAVID to perform GSEA - it is a user-friendly web-based tool

<div class="alert alert-block alert-warning">
<b>TODO:</b> supply more documentation/scripts for DAVID or another relevant GSEA. KEGG pathways is likely the best shot for working with non model organsims (as opposed to Gene Ontology). Provide user with information about choosing background gene sets. Consider choosing a program with user-defined gene sets?
</div>

# Plotting

##### Polar graphs (and linear, Manhattan-like...could be altered to scatter for more traditional Manhattan), Volcano plots, Top enrichment/depletion plots Individual guide enrichment/depletion bars along gene. See Andrew's code for Kmeans cluster plots. Consider using enrichplot for Pathway plots

> consider using enrichplot - https://yulab-smu.top/biomedical-knowledge-mining-book/enrichplot.html

<div class="alert alert-block alert-warning">
<b>NOTE:</b> See the few .ipynb notebook files for plotting. They require a csv file that has columns named 'locus', 'design', and 'Log2FC'

<b>TODO:</b> 
- quick subsetting function by p-value and log2FC to output 'FINAL' dataframe/csv for use or a couple different versions
- function for separating locus tag from gene name into separate values from the same string...or decide a standard way of setting up the library. (for example: A2754_2878107[c-]r)
- we could turn these notebooks into plotting functions instead to make the package more modular.
</div>
